In [119]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import glob
import lightgbm as lgb
from tqdm import tqdm

In [100]:
df = pd.read_csv('../blogfeedback/blogData_train.csv', header=None)
df_test = pd.concat(pd.read_csv(g,header=None) for g in glob.glob("../blogfeedback/blogData_test*"))

In [104]:
X_train = df[[x for x in range(276)]].to_numpy()
y_train = df[280].to_numpy()
#select
max_label = np.quantile(y_train, 0.9)
sel = y_train < max_label
y_train = y_train[sel]
X_train = X_train[sel]

In [105]:
X_test = df_test[[x for x in range(276)]].to_numpy()
y_test = df_test[280].to_numpy()
#select
sel = y_test < max_label
y_test = y_test[sel]
X_test = X_test[sel]

In [114]:
clf = lgb.LGBMRegressor(n_estimators=10000, max_depth=5, num_leaves=100, learning_rate=.1, )
clf.fit(X_train, y_train)
rmse_train = (((clf.predict(X_train) - y_train)**2).mean())**.5
rmse_test = (((clf.predict(X_test) - y_test)**2).mean())**.5

In [118]:
model = torch.nn.Sequential(
    torch.nn.Linear(276, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 1),
)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.999)


In [126]:
Xt = torch.tensor(X_train, dtype=torch.float32)
yt = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Xt, yt), batch_size=1000, shuffle=True)

bar = tqdm(range(1000))
for i in bar:
  for Xi, yi in dataloader:
    y_pred = model(Xi)
    loss = torch.nn.functional.mse_loss(y_pred, yi)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    scheduler.step()
    with torch.no_grad():
      rmse = (((y_pred - yi)**2).mean())**.5
    bar.set_description(f"loss: {loss.item():.3f}, rmse: {rmse.item():.3f}")

loss: 1.266, rmse: 1.125:  26%|██▋       | 265/1000 [02:20<06:29,  1.89it/s]


KeyboardInterrupt: 